## Project Intro: Intel Lab Data

This notebook contains information about data collected from 54 sensors deployed in the Intel Berkeley Research lab between February 28th and April 5th, 2004.

`Mica2Dot` sensors with weather boards collected timestamped topology information, along with humidity, temperature, light and voltage values once every 31 seconds.

Source: http://db.csail.mit.edu/labdata/labdata.html

## Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pyarrow as pa
import pyarrow.parquet as pq

## Dataset #1: External Historical Weather Data

In [5]:
weather_data = pd.read_json('data/weather.json')
weather_data.head()

,lat,lon,forecast
0,51.50853,-0.12574,"[{'forecast_dt': 1546300800, 'list': [{'main':..."
1,51.50853,-0.12574,"[{'forecast_dt': 1548979200, 'list': [{'main':..."
2,51.50853,-0.12574,"[{'forecast_dt': 1551657600, 'list': [{'main':..."
3,51.50853,-0.12574,"[{'forecast_dt': 1554336000, 'list': [{'main':..."
4,51.50853,-0.12574,"[{'forecast_dt': 1557014400, 'list': [{'main':..."


In [19]:
weather_data.shape

(6, 3)

In [25]:
weather_data.loc[0].forecast[0]['list']

[{'main': {'temp': 8.08,
   'pressure': 1034.848,
   'dew_point': 4.63,
   'grnd_pressure': 1029.908,
   'humidity': 78.61},
  'precipitation': None,
  'wind': {'speed': 2.959, 'deg': 292.075},
  'dt': 1546300800},
 {'main': {'temp': 6.34,
   'pressure': 1034.323,
   'dew_point': 4.6,
   'grnd_pressure': 1029.479,
   'humidity': 88.804},
  'precipitation': {'rate': 0,
   'fr_rain': 0,
   'convective': 0,
   'accumulated': 0,
   'ice': 0},
  'clouds': {'all': 66},
  'wind': {'speed': 3.277, 'deg': 280.693},
  'dt': 1546311600},
 {'main': {'temp': 5.77,
   'pressure': 1034.504,
   'dew_point': 4.15,
   'grnd_pressure': 1029.717,
   'humidity': 89.5},
  'precipitation': {'rate': 0, 'fr_rain': 0, 'convective': 0, 'ice': 0},
  'clouds': {'all': 33},
  'wind': {'speed': 3.575, 'deg': 286.864},
  'dt': 1546322400},
 {'main': {'temp': 6.02,
   'pressure': 1034.832,
   'dew_point': 4.45,
   'grnd_pressure': 1030.045,
   'humidity': 89.7},
  'precipitation': {'rate': 0,
   'fr_rain': 0,
   'conv

In [49]:
columns = ['datetime', 'external temp']
temp_list = []

for x in weather_data.loc[0].forecast[0]['list']:
    temp_list.append({'datetime': x['dt'], 'external temp': x['main']['temp']})

weather_df = pd.DataFrame(temp_list)
weather_df.head()

,datetime,external temp
0,1546300800,8.08
1,1546311600,6.34
2,1546322400,5.77
3,1546333200,6.02
4,1546344000,9.26


In [50]:
weather_df['datetime'] = pd.to_datetime(weather_df['datetime'])
weather_df['datetime'] = weather_df['datetime'].values.astype('<M8[s]')
weather_df.head()

,datetime,external temp
0,1970-01-01 00:00:01,8.08
1,1970-01-01 00:00:01,6.34
2,1970-01-01 00:00:01,5.77
3,1970-01-01 00:00:01,6.02
4,1970-01-01 00:00:01,9.26


## Dataset #2: Sensor Data

In [3]:
sensor_data = pd.read_csv('data/data.txt', sep=" ", header=None)
sensor_data.columns = ['date', 'time', 'epoch', 'mote_id', 'temperature', 'humidity', 'light', 'voltage']
sensor_data.head()

,date,time,epoch,mote_id,temperature,humidity,light,voltage
0,2004-03-31,03:38:15.757551,2,1.0,122.1530,-3.91901,11.04,2.03397
1,2004-02-28,00:59:16.02785,3,1.0,19.9884,37.09330,45.08,2.69964
2,2004-02-28,01:03:16.33393,11,1.0,19.3024,38.46290,45.08,2.68742
3,2004-02-28,01:06:16.013453,17,1.0,19.1652,38.80390,45.08,2.68742
4,2004-02-28,01:06:46.778088,18,1.0,19.1750,38.83790,45.08,2.69964


### Add datetime column

In [4]:
sensor_data['datetime'] = sensor_data['date'] + ' ' + sensor_data['time']
sensor_data['datetime'] = pd.to_datetime(sensor_data['datetime'])
sensor_data['datetime'] = sensor_data['datetime'].values.astype('<M8[s]')
sensor_data.head()

,date,time,epoch,mote_id,temperature,humidity,light,voltage,datetime
0,2004-03-31,03:38:15.757551,2,1.0,122.1530,-3.91901,11.04,2.03397,2004-03-31 03:38:15
1,2004-02-28,00:59:16.02785,3,1.0,19.9884,37.09330,45.08,2.69964,2004-02-28 00:59:16
2,2004-02-28,01:03:16.33393,11,1.0,19.3024,38.46290,45.08,2.68742,2004-02-28 01:03:16
3,2004-02-28,01:06:16.013453,17,1.0,19.1652,38.80390,45.08,2.68742,2004-02-28 01:06:16
4,2004-02-28,01:06:46.778088,18,1.0,19.1750,38.83790,45.08,2.69964,2004-02-28 01:06:46


### Inspect sensor data

In [5]:
sensor_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2313682 entries, 0 to 2313681
Data columns (total 9 columns):
date           object
time           object
epoch          int64
mote_id        float64
temperature    float64
humidity       float64
light          float64
voltage        float64
datetime       datetime64[ns]
dtypes: datetime64[ns](1), float64(5), int64(1), object(2)
memory usage: 158.9+ MB


In [6]:
sensor_data[['temperature', 'humidity', 'light', 'voltage']].describe()

,temperature,humidity,light,voltage
count,2.312781e+06,2.312780e+06,2.219804e+06,2.313156e+06
mean,3.920700e+01,3.390814e+01,4.072110e+02,2.492552e+00
std,3.741923e+01,1.732152e+01,5.394276e+02,1.795743e-01
min,-3.840000e+01,-8.983130e+03,0.000000e+00,9.100830e-03
25%,2.040980e+01,3.187760e+01,3.956000e+01,2.385220e+00
50%,2.243840e+01,3.928030e+01,1.582400e+02,2.527320e+00
75%,2.702480e+01,4.358550e+01,5.372800e+02,2.627960e+00
max,3.855680e+02,1.375120e+02,1.847360e+03,1.856000e+01


In [7]:
sensor_data[sensor_data.duplicated() == True].sum()

date           0.0
time           0.0
epoch          0.0
mote_id        0.0
temperature    0.0
humidity       0.0
light          0.0
voltage        0.0
datetime       0.0
dtype: float64

In [8]:
sensor_data.isna().sum()

date               0
time               0
epoch              0
mote_id          526
temperature      901
humidity         902
light          93878
voltage          526
datetime           0
dtype: int64

In [9]:
sensor_data.dropna(inplace=True)
sensor_data.isna().sum()

date           0
time           0
epoch          0
mote_id        0
temperature    0
humidity       0
light          0
voltage        0
datetime       0
dtype: int64

In [10]:
sensor_data.tail()

,date,time,epoch,mote_id,temperature,humidity,light,voltage,datetime
2313148,2004-04-02,09:06:21.488182,62421,58.0,24.1730,21.6575,1729.60,2.78836,2004-04-02 09:06:21
2313149,2004-04-02,09:07:22.173243,62423,58.0,24.1436,21.7310,1729.60,2.78836,2004-04-02 09:07:22
2313150,2004-04-02,09:07:52.31387,62424,58.0,24.1436,21.6575,1670.72,2.78836,2004-04-02 09:07:52
2313151,2004-04-02,09:08:52.189274,62426,58.0,24.1240,21.5839,1670.72,2.78836,2004-04-02 09:08:52
2313152,2004-04-02,09:09:22.206544,62427,58.0,24.1044,21.4367,1729.60,2.78836,2004-04-02 09:09:22


In [11]:
sensor_data.shape

(2219803, 9)

## Dataset #3: Sensor Location

In [12]:
sensor_locs = pd.read_csv('data/mote_locs.txt', sep=" ", header=None)
sensor_locs.head()

,0,1,2
0,1,21.5,23
1,2,24.5,20
2,3,19.5,19
3,4,22.5,15
4,5,24.5,12


In [13]:
sensor_locs.columns = ['mote_id', 'x_coord', 'y_coord']
sensor_locs.head()

,mote_id,x_coord,y_coord
0,1,21.5,23
1,2,24.5,20
2,3,19.5,19
3,4,22.5,15
4,5,24.5,12


In [14]:
sensor_locs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
mote_id    54 non-null int64
x_coord    54 non-null float64
y_coord    54 non-null int64
dtypes: float64(1), int64(2)
memory usage: 1.4 KB


In [15]:
sensor_locs['mote_id'] = sensor_locs['mote_id'].astype(float)
sensor_locs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 3 columns):
mote_id    54 non-null float64
x_coord    54 non-null float64
y_coord    54 non-null int64
dtypes: float64(2), int64(1)
memory usage: 1.4 KB


### Inspect sensor location data

In [16]:
sensor_locs['mote_id'].nunique()

54

In [17]:
sensor_locs.isna().sum()

mote_id    0
x_coord    0
y_coord    0
dtype: int64

In [18]:
sensor_locs.duplicated().sum()

0

### Combine dataset

In [19]:
sensor_signals = sensor_data.merge(sensor_locs, how='left', left_on='mote_id', right_on='mote_id')
sensor_signals.sort_values(['mote_id', 'datetime'], inplace=True)
sensor_signals.tail()

,date,time,epoch,mote_id,temperature,humidity,light,voltage,datetime,x_coord,y_coord
2215464,2004-04-04,16:21:06.290436,1611,58.0,23.9378,34.6403,86.48,2.59354,2004-04-04 16:21:06,NaN,NaN
2215465,2004-04-04,16:21:32.802969,1612,58.0,23.9280,34.6403,86.48,2.60491,2004-04-04 16:21:32,NaN,NaN
2215466,2004-04-04,20:54:11.616528,1613,58.0,21.6348,37.1963,3.22,2.56000,2004-04-04 20:54:11,NaN,NaN
2215467,2004-04-05,10:33:26.112499,1619,58.0,25.1040,33.6302,104.88,2.56000,2004-04-05 10:33:26,NaN,NaN
2215468,2004-04-05,10:33:51.519477,1620,58.0,25.1236,33.6302,86.48,2.56000,2004-04-05 10:33:51,NaN,NaN


In [20]:
sensor_signals.dropna(inplace=True)

In [21]:
sensor_signals.isna().sum()

date           0
time           0
epoch          0
mote_id        0
temperature    0
humidity       0
light          0
voltage        0
datetime       0
x_coord        0
y_coord        0
dtype: int64

In [22]:
sensor_signals['year'] = sensor_signals['datetime'].apply(lambda x: x.year)
sensor_signals['month'] = sensor_signals['datetime'].apply(lambda x: x.month)
sensor_signals.drop(['date', 'time', 'epoch'], axis=1, inplace=True)
sensor_signals.head()

,mote_id,temperature,humidity,light,voltage,datetime,x_coord,y_coord,year,month
1,1.0,19.9884,37.0933,45.08,2.69964,2004-02-28 00:59:16,21.5,23.0,2004,2
2,1.0,19.3024,38.4629,45.08,2.68742,2004-02-28 01:03:16,21.5,23.0,2004,2
3,1.0,19.1652,38.8039,45.08,2.68742,2004-02-28 01:06:16,21.5,23.0,2004,2
4,1.0,19.1750,38.8379,45.08,2.69964,2004-02-28 01:06:46,21.5,23.0,2004,2
5,1.0,19.1456,38.9401,45.08,2.68742,2004-02-28 01:08:45,21.5,23.0,2004,2


## Write to Parquet file

In [23]:
table = pa.Table.from_pandas(sensor_signals)

pq.write_to_dataset(
    table,
    root_path='output.parquet',
    partition_cols=['year', 'month', 'mote_id'],
)

/Users/rhoesada/anaconda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:114: FutureWarning: A future version of pandas will default to `skipna=True`. To silence this warning, pass `skipna=True|False` explicitly.
  result = infer_dtype(pandas_collection)
/Users/rhoesada/anaconda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:708: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels = getattr(columns, 'labels', None) or [
/Users/rhoesada/anaconda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:735: FutureWarning: the 'labels' keyword is deprecated, use 'codes' instead
  return pd.MultiIndex(levels=new_levels, labels=labels, names=columns.names)
/Users/rhoesada/anaconda/envs/py37/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels
